In [12]:
#this will allow conncetion to the data base 

from pony.orm import Database, db_session, Required
#This allows the code to communicate with the database.
db = Database(
   "mysql",
host="127.0.0.1",
user="root",
password="Kangroo13",
database="basecar",
)

#importing json , csv and xml 
import json
import csv 
import xml.etree.ElementTree as ET

json_file_path = 'user_data_sample.json'
xml_file_path = 'user_data_xml.xml'
csv_file_path = 'user_full_data.csv'
# a function that allows the  field names to come from Json 
def release_json_field_names(json_filename):
    with open(json_filename, 'r') as json_file:
        data = json.load(json_file)
    if isinstance(data, list):
        if data:
         sample_object = data[0]
        field_names = sample_object.keys()
    print(field_names)
            
# a function that allows the  field names to come from xml
def show_xml_field_names(xml_filename):
    tree = ET.parse (xml_filename)
    root = tree.getroot()
    if len(root) > 0:
            user_element = root[0]
            character_names = user_element.attrib.keys()
    print(character_names)
    
    
   # a function that allows the  field names to come from csv 
def print_csv_field_names(csv_filename):
    with open(csv_filename, 'r', newline='') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        if csv_reader.fieldnames:
            field_names = csv_reader.fieldnames
            print(field_names)
                  
release_json_field_names(json_file_path)
show_xml_field_names(xml_file_path)
print_csv_field_names(csv_file_path)

class records(db.Entity):
    first_name = Required(str)
    second_name = Required(str)
    age = Required(int)
    sex = Required(str)
    vehicle_make =Required(str)
    vehicle_model = Required(str)
    vehicle_year = Required (int)
    vehicle_type = Required(str)
    #mapping
    #command  to interact with the data base 
    
db.generate_mapping(create_tables=True)

#opens the csv file and inserts data in to tables 

@db_session
def read_csv_and_insert_into_db(csv_filename):
    with open(csv_filename, 'r',newline= '')as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            records(
                first_name=row['First Name'],
                second_name=row['Second Name'],
                age=int(row['Age (Years)']),
                sex= row['Sex'],
                vehicle_make=row['Vehicle Make'],
                vehicle_model=row['Vehicle Model'] ,
                vehicle_year=int(row['Vehicle Year']), 
                vehicle_type=row['Vehicle Type']      
                   
            )
               
read_csv_and_insert_into_db('user_full_data_.csv')


#this reads the json file and inserts it into the database 

def read_json_and_insert_into_db(json_filename):# there is also error handling involved 
    try:
        with open(json_filename, "r") as json_file:#opens the file 
            data = json.load(json_file)
            row_number = 1 
            for item in data : #the word item replaces row in json
                first_name=item['firstName'] #set out  every element of the class for the JSON data 
                last_name=item['lastName']
                age=item['age']
                person_correct = records.get(first_name = first_name, last_name = last_name, age= age)
                if person_correct:  #error hangling as notevery character has credit card and banking info
                    try: person_correct.set(
                        iban=item["iban"],
                        credit_card_number = item ["credir_card_number"],
                        credit_card_security_code=item ["credit_card_security_code"],
                        credit_card_start_date=item["credit_card_start_date"],
                        credit_card_end_date=item["credit_card_end_date"],
                        address_main=item["address_main"],
                        address_city=item["address_city"],
                        address_postcode=item["address_postcode"],
                        )
                    except ValueError:
                        print("ValueError at row number :", row_number, 'First name:', item['firstName'], "last name:", item['lastname'])
                    row_number=row_number +1
                else:
                    try:records(
                            first_name=item['firstName'],
                            last_name=item['lastName'],
                            age=item['age'],
                            iban=item["iban"],
                            credit_card_number = item ["credir_card_number"],
                            credit_card_security_code=item ["credit_card_security_code"],
                            credit_card_start_date=item["credit_card_start_date"],
                            credit_card_end_date=item["credit_card_end_date"],
                            address_main=item["address_main"],
                            address_city=item["address_city"],
                            address_postcode=item["address_postcode"],
                            )
                    except ValueError:
                        print("ValueError at row number :" , row_number, "First name : ", item['firstName'], "last name :", item['lastName'])
                    row_number = row_number +1
    except IOError:
        print("cant find file:", json_filename)

#finally after considering all the data the esception handling read the file one last time and place it in the database 

        read_json_and_insert_into_db('user_data.json')


        # the xml file data as we generated it already we cant use the simple basic code and  we need error hanling as well 
def read_xml_and_insert_into_db(xml_filename):
    try: #this is the begining of the error handling 
            tree = ET.parse(xml_filename)
            root= tree.getroot()
            row_number =1
            for user in root.findall('user'):
                first_name=user.get('firstName')#allows  all the users with a first name to be found 
                last_name=user.get('lastName')#allows all the users with a last name to be found
                age=user.get('age')# allows all the users withan age to be found
                person_correct = records.get(first_name=first_name,last_name=last_name,age=age)#this is  way of handling the fact that not every user had each element so if it doesnt else none
                if person_correct:
                                try:person_correct.set(
                                            sex=user.get('sex'),# if else loop esentially is used throughout this section
                                            retired=user.get('retired') == 'true' if user._get('retired') else None,
                                            dependants = int(user.get('dependants'))if user.get('dependants')else None,
                                            marital_status=user.get('marital_stats'),
                                            salary=int(user.get('salart')) if user.get('salary') else None,
                                            pension=int(user.get('pension'))if user.get('pension')else None,
                                            company=user.get('company'),
                                            comute_distance=float(user.get('commute_distance'))if user.get('commute_distance') else None,
                                            address_postcode=user.get('address_postcode')
                                        )
                                except ValueError:
                                        print("ValueError at row number :", row_number, "first name: ", user.get['firstName'], "last name: ", user.get['lastName'])
                                row_number = row_number +1
#print out the errors if any 
                else:
                    try:records(
                        first_name=user.get('firstName'),
                        last_name=user.get('lastName)'),
                        age=int(user.get('age'))if user.get('age')else None,
                        sex=user.get('sex'),
                        retired=user.get('retired')=='True' if user.get('retired')else None,
                        dependants = int(user.get('dependants')) if user.get('dependants')else None,
                        marital_status=user.get('marital_status'),
                        salary=int(user.get('salary'))if user.get('salary')else None,
                        pension=int(user.get('pension'))if user.get('pension')else None,
                        company=user.get('company'),
                        commute_distance=float(user.get('commute_distance'))if user.get('commute_distance')else None, 
                        address_postcode = user.get('address_postcode')  )
                    except KeyError:
                        print("KeyError at row number :", row_number, "First name:", user.get('firstName'), "Last name:",user.get('lastName'))
    except IOError:
        print ("cant find file :", xml_filename)

read_xml_and_insert_into_db(user_data_xml.xml)

      #else get all the elements of the lass records and place it in to the database 

  
















read_json_and_insert_into_db_json('user_data.json')
    

                 
                


                      

dict_keys(['firstName', 'lastName', 'age', 'iban', 'credit_card_number', 'credit_card_security_code', 'credit_card_start_date', 'credit_card_end_date', 'address_main', 'address_city', 'address_postcode'])
dict_keys(['firstName', 'lastName', 'age', 'sex', 'retired', 'dependants', 'marital_status', 'salary', 'pension', 'company', 'commute_distance', 'address_postcode'])
['First Name', 'Second Name', 'Age (Years)', 'Sex', 'Vehicle Make', 'Vehicle Model', 'Vehicle Year', 'Vehicle Type']


ERDiagramError: Entity class name should start with a capital letter. Got: records